In [1]:
import csv
import glob
import json
import os
import time

import numpy as np
import pandas as pd
from matpower import start_instance
from matpowercaseframes import CaseFrames
from oct2py.core import Oct2PyError

from pypglib import PATH_PYPGLIB

In [2]:
output_dir = "../bench"
output_dir_opf = os.path.join(output_dir, "opf")

os.makedirs(output_dir_opf, exist_ok=True)

In [3]:
m = start_instance()

In [4]:
def dump_sol_to_json(sol, output_dir, file_name):
    def convert_ndarray(obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        raise TypeError(f"Object of type {type(obj)} is not JSON serializable")

    json_file_path = os.path.join(output_dir, f"{file_name}.json")

    with open(json_file_path, "w") as json_file:
        json.dump(sol, json_file, indent=4, default=convert_ndarray)


def extract_sol(mpc, m):
    # NOTE: too big if pull rundcopf data
    #   m.eval("_r1.raw = rmfield(_r1.raw, 'task');")
    #   m.eval("_r1 = rmfield(_r1, 'om');")
    #   sol = m.pull("_r1")

    sol = {
        "baseMVA": mpc.baseMVA,
        "version": mpc.version,
        "bus": m.eval("_r1.bus;", verbose=False),
        "gen": m.eval("_r1.gen;", verbose=False),
        "branch": m.eval("_r1.branch;", verbose=False),
        "gencost": mpc.gencost,
        "success": m.eval("_r1.success;", verbose=False),
    }
    return sol


def run_ed(sol, mpopt):
    # TODO: check at least voltages and power constraints
    if sol["success"] == 1:
        sol_ed = m.runpf(sol, mpopt, verbose=False)
        if sol_ed["success"] == 1:
            status = "Success"
            cost = m.totcost(sol_ed["gencost"], sol_ed["gen"][:, [PG]]).sum()
        else:
            status = "Not Converge"
            cost = "--"
    else:
        sol_ed = {}
        status = "Infeasible"
        cost = "inf"  # Infeasible solution
    return sol_ed, status, cost

In [5]:
PG = 1  # index for PG
opf_dir = os.path.join(PATH_PYPGLIB, "opf")
mpopt = m.mpoption("verbose", 0, "out.all", 0)
m.push("_mpopt", mpopt)

csv_file_path = os.path.join(output_dir_opf, "opf_benchmark_results.csv")
fieldnames = [
    "file_name",
    "matpower-pip_status",
    "matpower-pip_cost",
    "matpower-pip_python_time_perf",
]

# create results and check already run case
if not os.path.isfile(csv_file_path):
    with open(csv_file_path, mode="w", newline="") as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
df_results = pd.read_csv(csv_file_path)
file_names_m = df_results["file_name"].tolist()

file_paths = list(glob.glob(os.path.join(opf_dir, "**", "*.m"), recursive=True))
for file_path in file_paths:
    file_name = os.path.basename(file_path)
    if file_name in file_names_m:
        continue

    mpc = m.loadcase(file_path)
    try:
        start_time_perf = time.perf_counter()
        m.push("_mpc", mpc)
        m.eval("_r1 = runopf(_mpc, _mpopt);", verbose=False)
        sol = extract_sol(mpc, m)
        python_time_perf = time.perf_counter() - start_time_perf

        sol_ed, status, cost = run_ed(sol, mpopt)
    except Oct2PyError as e:
        print(e)
        print(file_name)
        sol = {}
        status = "Error"
        cost = "Err"
        python_time_perf = np.inf

    # NOTE: json is too big, 5 MB for single case
    # dump_sol_to_json(sol, output_dir_opf, file_name)  # too big

    try:
        base_name = os.path.splitext(os.path.basename(file_name))[0]
        cf = CaseFrames(sol)
        cf.gen.to_csv(os.path.join(output_dir_opf, f"{base_name}.gen.csv"))
        # cf.bus.to_csv(
        #     os.path.join(output_dir_opf, f"{base_name}.bus.csv")
        # )
    except AttributeError:
        # errors
        pass

    with open(csv_file_path, mode="a", newline="") as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writerow(
            {
                "file_name": file_name,
                "matpower-pip_status": status,
                "matpower-pip_cost": cost,
                "matpower-pip_python_time_perf": python_time_perf,
            }
        )

In [ ]:
PG = 1  # index for PG
opf_dir = os.path.join(PATH_PYPGLIB, "opf")
mpopt = m.mpoption("verbose", 0, "out.all", 0)
m.push("_mpopt", mpopt)

csv_file_path = os.path.join(output_dir_opf, "dcopf_benchmark_results.csv")
fieldnames = [
    "file_name",
    "matpower-pip_dcopf_status",
    "matpower-pip_dcopf_cost",
    "matpower-pip_dcopf_python_time_perf",
]

# create results and check already run case
if not os.path.isfile(csv_file_path):
    with open(csv_file_path, mode="w", newline="") as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
df_results = pd.read_csv(csv_file_path)
file_names_m = df_results["file_name"].tolist()

file_paths = list(glob.glob(os.path.join(opf_dir, "**", "*.m"), recursive=True))
for file_path in file_paths:
    file_name = os.path.basename(file_path)
    if file_name in file_names_m:
        continue

    mpc = m.loadcase(file_path)
    try:
        start_time_perf = time.perf_counter()
        m.push("_mpc", mpc)
        m.eval("_r1 = rundcopf(_mpc, _mpopt);", verbose=False)
        sol = extract_sol(mpc, m)
        python_time_perf = time.perf_counter() - start_time_perf

        sol_ed, status, cost = run_ed(sol, mpopt)

    except Oct2PyError as e:
        print(e)
        print(file_name)
        sol = {}
        status = "Error"
        cost = "Err"
        python_time_perf = np.inf

    # NOTE: json is too big, 5 MB for single case
    # dump_sol_to_json(sol, output_dir_opf, file_name)  # too big

    try:
        base_name = os.path.splitext(os.path.basename(file_name))[0]
        cf = CaseFrames(sol)
        cf.gen.to_csv(os.path.join(output_dir_opf, f"{base_name}_dcopf.gen.csv"))
        # cf.bus.to_csv(
        #     os.path.join(output_dir_opf, f"{base_name}.bus.csv")
        # )
    except AttributeError:
        # errors
        pass

    with open(csv_file_path, mode="a", newline="") as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writerow(
            {
                "file_name": file_name,
                "matpower-pip_dcopf_status": status,
                "matpower-pip_dcopf_cost": cost,
                "matpower-pip_dcopf_python_time_perf": python_time_perf,
            }
        )

In [ ]:
os.makedirs(output_dir_opf, exist_ok=True)
df_results = pd.read_csv(csv_file_path)

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)

print(df_results)

                   file_name matpower-pip_status  matpower-pip_cost  matpower-pip_python_time_perf
0   pglib_opf_case4837_goc.m             Success       8.722553e+05                      23.271484
1  pglib_opf_case2746wop_k.m             Success       1.208259e+06                      14.201063
2   pglib_opf_case4601_goc.m             Success       8.262416e+05                      18.952148


In [ ]:
# import scipy.io
# import julia
# from julia import Main

# # Initialize Julia and load the required packages
# julia.Julia(compiled_modules=False)
# Main.eval(
# """
# using JuMP, PowerModels, Ipopt
# solver = optimizer_with_attributes(Ipopt.Optimizer, "tol"=>1e-6)
# """
# )

# # List of case files to run (replace with actual .mat file paths or data objects)
# case_files = ["case1.mat", "case2.mat", "case3.mat"]

# # To store the results
# results = []

# # Loop through each case and run the optimization
# for case_file in case_files:
#     # Load the .mat file using scipy.io in Python
#     mat_data = scipy.io.loadmat(case_file)

#     # Pass the .mat data to Julia
#     Main.mat_data = mat_data

#     # Run the AC Optimal Power Flow in Julia and capture the result
#     result = Main.eval('run_ac_opf(mat_data, solver)')

#     # Store the result in the Python results list
#     results.append(result)

# # Now `results` contains the optimization results for each case
# for i, res in enumerate(results):
#     print(f"Result for case {i+1}: {res}")